In [1]:
"""
 Simple Multi-Layer Pecptron with MNIST
 @Sungjoon Choi (sungjoon.choi@cpslab.snu.ac.kr)
"""
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.examples.tutorials.mnist import input_data

%matplotlib inline  
print ("Packs loaded.")

Packs loaded.


In [2]:
# MNIST
mnist = input_data.read_data_sets('data/', one_hot=True)

Extracting data/train-images-idx3-ubyte.gz
Extracting data/train-labels-idx1-ubyte.gz
Extracting data/t10k-images-idx3-ubyte.gz
Extracting data/t10k-labels-idx1-ubyte.gz


In [3]:
# Parameters
learning_rate   = 0.001
training_epochs = 10
batch_size      = 100
display_step    = 1

# Network Parameters
n_hidden_1 = 256 # 1st layer num features
n_hidden_2 = 128 # 2nd layer num features
n_input    = 784 # MNIST data input (img shape: 28*28)
n_classes  = 10 # MNIST total classes (0-9 digits)

# tf Graph input
x = tf.placeholder("float", [None, n_input])
y = tf.placeholder("float", [None, n_classes])

# Create model
def multilayer_perceptron(_X, _weights, _biases):
    layer_1 = tf.nn.sigmoid(tf.add(tf.matmul(_X, _weights['h1']), _biases['b1'])) 
    layer_2 = tf.nn.sigmoid(tf.add(tf.matmul(layer_1, _weights['h2']), _biases['b2']))
    return (tf.sigmoid(tf.matmul(layer_2, _weights['out']) + _biases['out']))
    # return (tf.sigmoid(tf.matmul(layer_2, _weights['out']) + _biases['out']))
    
# Store layers weight & bias
stddev = 0.1 # <== This greatly affects accuracy!! 
weights = {
    'h1': tf.Variable(tf.random_normal([n_input, n_hidden_1], stddev=stddev)),
    'h2': tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2], stddev=stddev)),
    'out': tf.Variable(tf.random_normal([n_hidden_2, n_classes], stddev=stddev))
}
biases = {
    'b1': tf.Variable(tf.random_normal([n_hidden_1])),
    'b2': tf.Variable(tf.random_normal([n_hidden_2])),
    'out': tf.Variable(tf.random_normal([n_classes]))
}
print ("Network Ready")

Network Ready


In [4]:
# Construct model
pred = multilayer_perceptron(x, weights, biases)

# Define loss and optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(pred, y)) # Softmax loss
optm = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost) # Adam Optimizer
corr = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))    
accr = tf.reduce_mean(tf.cast(corr, "float"))

# Initializing the variables
init = tf.initialize_all_variables()

In [8]:
# Launch the graph
sess = tf.Session()
sess.run(init)

# Training cycle
for epoch in range(training_epochs):
    avg_cost = 0.
    total_batch = int(mnist.train.num_examples/batch_size)
    # Loop over all batches
    for i in range(total_batch):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        # Fit training using batch data
        sess.run(optm, feed_dict={x: batch_xs, y: batch_ys})
        # Compute average loss
        avg_cost += sess.run(cost, feed_dict={x: batch_xs, y: batch_ys})/total_batch
    # Display logs per epoch step
    if epoch % display_step == 0:
        print ("Epoch: %03d/%03d cost: %.9f" % (epoch, training_epochs, avg_cost))
        train_acc = sess.run(accr, feed_dict={x: batch_xs, y: batch_ys})
        print ("Training accuracy: %.3f" % (train_acc))
        test_acc = sess.run(accr, feed_dict={x: mnist.test.images, y: mnist.test.labels})
        print ("Test accuracy: %.3f" % (test_acc))

print ("Optimization Finished!")


Epoch: 000/010 cost: 1.710917208
Training accuracy: 0.930
Test accuracy: 0.911
Epoch: 001/010 cost: 1.544890469
Training accuracy: 0.930
Test accuracy: 0.933
Epoch: 002/010 cost: 1.520105678
Training accuracy: 0.960
Test accuracy: 0.945
Epoch: 003/010 cost: 1.506742131
Training accuracy: 0.960
Test accuracy: 0.954
Epoch: 004/010 cost: 1.497843306
Training accuracy: 0.970
Test accuracy: 0.959
Epoch: 005/010 cost: 1.491715848
Training accuracy: 0.990
Test accuracy: 0.963
Epoch: 006/010 cost: 1.486772999
Training accuracy: 0.960
Test accuracy: 0.966
Epoch: 007/010 cost: 1.483362348
Training accuracy: 1.000
Test accuracy: 0.970
Epoch: 008/010 cost: 1.480253498
Training accuracy: 0.990
Test accuracy: 0.971
Epoch: 009/010 cost: 1.477486276
Training accuracy: 1.000
Test accuracy: 0.973
Optimization Finished!


In [6]:
test_acc = sess.run(accr, feed_dict={x: mnist.test.images, y: mnist.test.labels})
print ("Training accuracy: %.3f" % (test_acc))

Training accuracy: 0.973
